In [ ]:
import cv2
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import fashion_mnist
from keras.applications import DenseNet121

In [ ]:
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

X_train = np.array([cv2.resize(im, (128, 128)) for im in X_train])
X_test = np.array([cv2.resize(im, (128, 128)) for im in X_test])

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train /= 255.0
X_test /= 255.0

X_train = np.stack((X_train,)*3, axis=-1)
X_test = np.stack((X_test,)*3, axis=-1)

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [ ]:
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras import layers, models

base_model = DenseNet121(include_top=False, weights='imagenet', input_shape=(128, 128, 3))
base_model.trainable = False

x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dense(10, activation='softmax')(x)

model = models.Model(inputs=base_model.input, outputs=x)

In [ ]:
from keras.optimizers import Adam

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
from keras.callbacks import EarlyStopping

batch_size = 32
epochs = 10

early_stopping = EarlyStopping(
    monitor='accuracy',
    patience = 1,
    restore_best_weights = True
)

history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, callbacks=[early_stopping])

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print(f'\nTest accuracy: {test_acc}')

In [ ]:
model.save('NN/model_fashion_mnist_densenet.h5')